In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [76]:
title_read="""Matter-Wave Bright Solitons with a Finite Background in Spinor
  Bose-Einstein Condensates
"""

In [77]:
RES_ARTICLES="..\data\project_data.csv"
# ARTICLE_READ=[1,10]
num_recommendations=5

In [78]:
res_articles = pd.read_csv(RES_ARTICLES,encoding='utf-8',index_col=0)
#drop all the unnamed columns
res_articles.drop(res_articles.columns[res_articles.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
res_articles.head()

,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,...,authors_parsed/17/3,authors_parsed/18/3,authors_parsed/19/3,authors_parsed/20/3,authors_parsed/21/3,authors_parsed/22/3,authors_parsed/23/3,authors_parsed/24/3,authors_parsed/25/3,authors_parsed/26/3
id,,,,,,,,,,,,,,,,,,,,,
0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,NaN,A fully differential calculation in perturba...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,NaN,NaN,NaN,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",NaN,NaN,NaN,physics.gen-ph,NaN,The evolution of Earth-Moon system is descri...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,NaN,NaN,NaN,math.CO,NaN,We show that a determinant of Stirling cycle...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,NaN,"Illinois J. Math. 52 (2008) no.2, 681-689",NaN,NaN,math.CA math.FA,NaN,In this paper we show how to compute the $\L...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
res_ids = res_articles.index.tolist()
res_ids

['0704.0001',
 '0704.0002',
 '0704.0003',
 '0704.0004',
 '0704.0005',
 '0704.0006',
 '0704.0007',
 '0704.0008',
 '0704.0009',
 '0704.0010',
 '0704.0011',
 '0704.0012',
 '0704.0013',
 '0704.0014',
 '0704.0015',
 '0704.0016',
 '0704.0017',
 '0704.0018',
 '0704.0019',
 '0704.0020',
 '0704.0021',
 '0704.0022',
 '0704.0023',
 '0704.0024',
 '0704.0025',
 '0704.0026',
 '0704.0027',
 '0704.0028',
 '0704.0029',
 '0704.0030',
 '0704.0031',
 '0704.0032',
 '0704.0033',
 '0704.0034',
 '0704.0035',
 '0704.0036',
 '0704.0037',
 '0704.0038',
 '0704.0039',
 '0704.0040',
 '0704.0041',
 '0704.0042',
 '0704.0043',
 '0704.0044',
 '0704.0045',
 '0704.0046',
 '0704.0047',
 '0704.0048',
 '0704.0049',
 '0704.0050',
 '0704.0051',
 '0704.0052',
 '0704.0053',
 '0704.0054',
 '0704.0055',
 '0704.0056',
 '0704.0057',
 '0704.0058',
 '0704.0059',
 '0704.0060',
 '0704.0061',
 '0704.0062',
 '0704.0063',
 '0704.0064',
 '0704.0065',
 '0704.0066',
 '0704.0067',
 '0704.0068',
 '0704.0069',
 '0704.0070',
 '0704.0071',
 '0704

In [80]:
res_articles = res_articles[['title','abstract']].dropna()
#articles is a list of all articles
titles = res_articles['title'].tolist()
titles[0] #an uncleaned article

'Calculation of prompt diphoton production cross sections at Tevatron and\r\n  LHC energies'

In [81]:
titles=[title_read]+titles

In [82]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(titles)

In [83]:
lsa_model = TruncatedSVD(n_components=100)  # Choose the desired number of components
lsa_matrix = lsa_model.fit_transform(tfidf_matrix)

In [84]:
similarity_matrix = cosine_similarity(lsa_matrix)

In [85]:
similarity_matrix

array([[ 1.        , -0.04245868,  0.01252951, ..., -0.0244124 ,
        -0.02652898, -0.00165853],
       [-0.04245868,  1.        ,  0.02102401, ...,  0.06609756,
         0.09107146, -0.01964896],
       [ 0.01252951,  0.02102401,  1.        , ...,  0.19479781,
         0.08415533, -0.059979  ],
       ...,
       [-0.0244124 ,  0.06609756,  0.19479781, ...,  1.        ,
         0.42391596, -0.0632088 ],
       [-0.02652898,  0.09107146,  0.08415533, ...,  0.42391596,
         1.        ,  0.13320453],
       [-0.00165853, -0.01964896, -0.059979  , ..., -0.0632088 ,
         0.13320453,  1.        ]])

In [86]:
input_text_index = 0  # Index of the input text
similar_text_indices = similarity_matrix[input_text_index].argsort()[-(num_recommendations+1):][::-1][1:]  # Exclude the input text

In [87]:
ref_indices = similar_text_indices

In [88]:
similar_text_indices = [res_ids[i-2] for i in similar_text_indices]


In [89]:
similar_text_indices

['0704.0145', '0704.0812', '0704.0121', '0704.0385', '0704.0887']

In [91]:
recommended_texts = [titles[idx-1] for idx in ref_indices]

In [ ]:
recommended_texts

['Singularity Resolution in Isotropic Loop Quantum Cosmology: Recent\r\n  Developments',
 'The Na I D resonance lines in main sequence late-type stars',
 'Meta-Stable Brane Configuration of Product Gauge Groups',
 'Super-shell structures and pairing in ultracold trapped Fermi gases',
 'Non-extensive thermodynamics of 1D systems with long-range interaction']